## Задача
Необходимо создать модель, которая принимает на вход текст документа и наименование одного из двух пунктов, и возвращает возвращать соответствующий кусочек текста из текста документа (фрагмент, начало индекса фрагмента, конец индекса фрагмента)

Пример
```
    {'text': ['Размер обеспечения исполнения контракта 6593.25 Российский рубль'], [1279], [1343] }
```

In [10]:
import json
from sklearn.utils import shuffle

import numpy as np
import pandas as pd
import ast
import random

import spacy
from spacy.training.example import Example
from tqdm import tqdm
from spacy.tokens import DocBin
from spacy.util import filter_spans
from spacy.lang.ru import Russian
from spacy.pipeline.spancat import DEFAULT_SPANCAT_MODEL
from spacy.pipeline.spancat import DEFAULT_SPANCAT_SINGLELABEL_MODEL

from sklearn.model_selection import train_test_split

In [2]:
with open('dataset\\train.json', 'r', encoding='utf-8') as f: #открыли файл с данными
    data = json.load(f) #загнали все, что получилось в переменную
data[0]

{'id': 809436509,
 'text': 'Извещение о проведении открытого конкурса в электронной форме для закупки №0328300032822000806 Общая информация Номер извещения 0328300032822000806 Наименование объекта закупки Поставка продуктов питания Способ определения поставщика (подрядчика, исполнителя) Открытый конкурс в бль Порядок внесения денежных средств в качестве обеспечения заявки на участие в закупке, а также условия гарантии Обеспечение заявки на участие в закупке может предоставляться участником закупки в виде денежных средств или независимой гарантии, предусмотренной ст. 45 Федерального закона № 44-ФЗ. Выбор способа обеспечения осуществляется участником закупки самостоятельно. Срок действия независимой гарантии должен составлять не менее месяца с даты окончания срока подачи заявок. Обеспечение заявки на участие в закупке предоставляется в соответствии с ч. 5 ст. 44 Федерального закона № 44-ФЗ. Условия независимой гарантии в соответствии со ст. 45 Федерального закона № 44-ФЗ. Реквизиты счета

In [3]:
max_word = 0
min_word = 10000
for dict in data:
    answer_start = dict.get('extracted_part').get('answer_start')[0]
    answer_end = dict.get('extracted_part').get('answer_end')[0]
    if answer_start != 0 or answer_end != 0:
        lenght = len(dict.get('extracted_part').get('text')[0].split(' '))
        if max_word<lenght:
            max_word = lenght
        if min_word > lenght:
            min_word = lenght
print(min_word)
print(max_word)

4
40


### Создаём датасет для SpanCategorizer


In [9]:
#"обеспечение исполнения контракта" EOC
#"обеспечение гарантийных обязательств" PWO

span_key = "sc"
def preprocessing(data):
    data_train = {'classes' : ['обеспечение исполнения контракта', "обеспечение гарантийных обязательств"], 'annotations' : []}
    data_missing = []
    max_lenght = 0
    for dict in data:
        text = dict.get('text')
        label = dict.get('label')
        answer_start = dict.get('extracted_part').get('answer_start')[0]
        answer_end = dict.get('extracted_part').get('answer_end')[0]
        if answer_start == 0 and answer_end == 0:
            data_missing.append((text, {'spans': []}))
        else:
            data_train['annotations'].append({'text': text, 'spans': {span_key: [(answer_start, answer_end, label)]}})

            #row = Example.from_dict(nlp.make_doc(text), {'spans': {span_key: [(answer_start, answer_end, label)]}})
            #row = (text, {'spans': {span_key: [(answer_start, answer_end, label)]}})
            #data_train.append(row)

    return data_train, data_missing

data_train, data_missing = preprocessing(data)
data_train['annotations'][0]

{'text': 'Извещение о проведении открытого конкурса в электронной форме для закупки №0328300032822000806 Общая информация Номер извещения 0328300032822000806 Наименование объекта закупки Поставка продуктов питания Способ определения поставщика (подрядчика, исполнителя) Открытый конкурс в бль Порядок внесения денежных средств в качестве обеспечения заявки на участие в закупке, а также условия гарантии Обеспечение заявки на участие в закупке может предоставляться участником закупки в виде денежных средств или независимой гарантии, предусмотренной ст. 45 Федерального закона № 44-ФЗ. Выбор способа обеспечения осуществляется участником закупки самостоятельно. Срок действия независимой гарантии должен составлять не менее месяца с даты окончания срока подачи заявок. Обеспечение заявки на участие в закупке предоставляется в соответствии с ч. 5 ст. 44 Федерального закона № 44-ФЗ. Условия независимой гарантии в соответствии со ст. 45 Федерального закона № 44-ФЗ. Реквизиты счета в соответствии с 

In [21]:
label_json = {"labels": {"spancat": ["обеспечение исполнения контракта", "обеспечение гарантийных обязательств"]}}
with open('labels.json', 'w', encoding='utf-8') as outfile:
    json.dump(label_json, outfile, ensure_ascii=False,)

In [12]:
nlp = spacy.blank("ru") # load a new spacy model
doc_bin = DocBin()

In [17]:
for training_example  in tqdm(data_train['annotations']):
    text = training_example['text']
    labels = training_example['spans']
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in labels[span_key]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents
    doc_bin.add(doc)

doc_bin.to_disk("training_data_spans.spacy") # save the docbin object

100%|██████████| 1492/1492 [00:02<00:00, 730.30it/s]


In [14]:
!python -m spacy init fill-config base_config.cfg config.cfg

[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [15]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data_spans.spacy --paths.dev ./training_data_spans.spacy --gpu-id 0

[i] Saving to output directory: .
[i] Using GPU: 0

=========================== Initializing pipeline ===========================


[2023-04-11 10:38:36,098] [INFO] Set up nlp object from config
[2023-04-11 10:38:36,108] [INFO] Pipeline: ['tok2vec', 'spancat']
[2023-04-11 10:38:36,111] [INFO] Created vocabulary
Traceback (most recent call last):
  File "D:\Jupiter\venv\lib\site-packages\spacy\language.py", line 1286, in initialize
    init_vocab(
  File "D:\Jupiter\venv\lib\site-packages\spacy\training\initialize.py", line 132, in init_vocab
    load_vectors_into_model(nlp, vectors)
  File "D:\Jupiter\venv\lib\site-packages\spacy\training\initialize.py", line 153, in load_vectors_into_model
    vectors_nlp = load_model(name, vocab=nlp.vocab, exclude=exclude)
  File "D:\Jupiter\venv\lib\site-packages\spacy\util.py", line 449, in load_model
    raise IOError(Errors.E050.format(name=name))
OSError: [E050] Can't find model 'en_core_web_lg'. It doesn't seem to be a Python package or a valid path to a data directory.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):


In [4]:
#"обеспечение исполнения контракта" EOC
#"обеспечение гарантийных обязательств" PWO

span_key = "sc"
def preprocessing(data):
    nlp = spacy.blank('ru')
    data_train = []
    data_missing = []
    max_lenght = 0
    for dict in data:
        text = dict.get('text')
        label = dict.get('label')
        answer_start = dict.get('extracted_part').get('answer_start')[0]
        answer_end = dict.get('extracted_part').get('answer_end')[0]
        if answer_start == 0 and answer_end == 0:
            data_missing.append((text, {'spans': []}))
        else:
            row = (text, {'spans': {span_key: [(answer_start, answer_end, label)]}})
            #row = Example.from_dict(nlp.make_doc(text), {'spans': {span_key: [(answer_start, answer_end, label)]}})
            data_train.append(row)

    return data_train, data_missing

data_train, data_missing = preprocessing(data)
data_train, data_valid = train_test_split(data_train, random_state=12345, test_size=0.25)
#data_valid+=data_missing
data_train = data_train[0:2]
'''
data_train, data_valid = train_test_split(preprocessing(data), random_state=12345, test_size=0.25)
print(len(data_train))
print(len(data_valid))
data_train[0]
'''

print(len(data_train))
print(len(data_valid))
data_train[0]

2
373


('35 УТВЕРЖДАЮ Ломакина Наталья Владимировна Директор МБУ "МФЦ городского округа Балашиха" «01» сентября 2022г. ОРМЕ» документации, а именно: 1 час с момента размещения оператором электронной площадки протокола сопоставления ценовых предложений. 26. Дата начала и окончания срока рассмотрения вторых частей заявок на участие в аукционе в электронной форме Дата начала срока рассмотрения вторых частей заявок на участие в аукционе в электронной форме: «12» сентября 2022 Дата окончания срока рассмотрения вторых частей заявок на участие в аукционе в электронной форме: «12» сентября 2022 27. Дата подведения итогов аукциона в электронной форме Дата подведения итогов аукциона в электронной форме: «12» сентября 2022 28. Обеспечение заявок на участие в аукционе в электронной форме Не требуется 28.1. Размер обеспечения заявок на участие в аукционе в электронной форме Не требуется 28.2. Срок и порядок предоставления обеспечения заявок на участие в аукционе в электронной форме Не требуется 29. Обеспе

In [8]:
spacy.require_gpu()
print(spacy.prefer_gpu())

#nlp = spacy.blank("ru")
nlp = spacy.load("ru_core_news_sm")
pipe_exceptions = ["spancat_singlelabel"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

config = {
    #this refers to the minimum probability to consider a prediction positive
    #"threshold": 0.5,
    #the span key refers to the key in doc.spans
    "spans_key": span_key,
    #this refers to the maximum number of labels to consider positive per span
    #"max_positive": None,
    #a model instance that is given a list of documents with start end indices representing the labelled spans
    "model": DEFAULT_SPANCAT_SINGLELABEL_MODEL, # DEFAULT_SPANCAT_MODEL
    #A function that suggests spans. This suggester is fixed n-gram length of up to 3 tokens
    #"suggester": {"@misc": "spacy.ngram_suggester.v1", "sizes": [1, 2, 3]},
    "suggester": {"@misc": "spacy.ngram_range_suggester.v1", "min_size": 3, "max_size": 40},
}


span = nlp.add_pipe("spancat_singlelabel", config=config) # spancat_singlelabel

span.add_label("обеспечение исполнения контракта")
span.add_label("обеспечение гарантийных обязательств")

#span.add_label("EOC")
#span.add_label("PWO")

#optimizer = nlp.begin_training()

nlp.initialize()
optimizer = nlp.create_optimizer()

with nlp.disable_pipes(*unaffected_pipes):
    for epoch in range(20):
        random.shuffle(data_train)
        loss = {}
        for batch in spacy.util.minibatch(data_train, size=2):
            example = None
            for text, annotations in batch:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations, drop=0.2)

            nlp.update([example], sgd=optimizer, losses=loss)
        print('Epoch: ',epoch, ' losses: ', loss)

#nlp.to_disk("model")

doc = nlp(data_train[0][0])
print(doc.spans)

True
Epoch:  0  losses:  {'spancat_singlelabel': 8904.6669921875}
Epoch:  1  losses:  {'spancat_singlelabel': 7750.1533203125}
Epoch:  2  losses:  {'spancat_singlelabel': 6651.8671875}
Epoch:  3  losses:  {'spancat_singlelabel': 5450.0908203125}
Epoch:  4  losses:  {'spancat_singlelabel': 4270.05615234375}
Epoch:  5  losses:  {'spancat_singlelabel': 3681.069091796875}
Epoch:  6  losses:  {'spancat_singlelabel': 2614.10009765625}
Epoch:  7  losses:  {'spancat_singlelabel': 1630.5703125}
Epoch:  8  losses:  {'spancat_singlelabel': 1182.291259765625}
Epoch:  9  losses:  {'spancat_singlelabel': 757.8826293945312}
Epoch:  10  losses:  {'spancat_singlelabel': 473.1317443847656}
Epoch:  11  losses:  {'spancat_singlelabel': 289.82464599609375}
Epoch:  12  losses:  {'spancat_singlelabel': 170.41127014160156}
Epoch:  13  losses:  {'spancat_singlelabel': 102.01383209228516}
Epoch:  14  losses:  {'spancat_singlelabel': 63.14659881591797}
Epoch:  15  losses:  {'spancat_singlelabel': 36.215160369873

In [ ]:
def accuracy(true, pred):
    true_pred = 0
    for i in range(len(true)):
        if true[i] == pred[i]:
            true_pred+=1
    return true_pred/len(true)

In [ ]:
def create_true_pred_arr(data, model):
    data_true = []
    data_pred = []
    for i in range(len(data)):
        if data[i][1].get('entities'):
            left = data[i][1].get('entities')[0][0]
            right = data[i][1].get('entities')[0][1]
            text = data[i][0]
            data_true.append(text[left:right])
        else:
            #print(data[i][1].get('entities'))
            data_true.append('')

        doc = model(data[i][0])
        print(doc.ents)
        if list(doc.ents):
            data_pred.append(str(doc.ents[0]))
        else:
            data_pred.append('')
    print(len(data_true))
    print(len(data_pred))
    return data_true, data_pred

#data_true, data_pred = create_true_pred_arr(data_valid, nlp)
#print(len(data_true))
#print(len(data_pred))

In [ ]:
data_true, data_pred = create_true_pred_arr(data_train, nlp)
print('Метрика accuracy на тренировочной выборке:', accuracy(data_true, data_pred))

data_true, data_pred = create_true_pred_arr(data_valid, nlp)
print('Метрика accuracy на тестовой выборке:', accuracy(data_true, data_pred))

In [8]:
data_train[0][0]

[('35 УТВЕРЖДАЮ Ломакина Наталья Владимировна Директор МБУ "МФЦ городского округа Балашиха" «01» сентября 2022г. ОРМЕ» документации, а именно: 1 час с момента размещения оператором электронной площадки протокола сопоставления ценовых предложений. 26. Дата начала и окончания срока рассмотрения вторых частей заявок на участие в аукционе в электронной форме Дата начала срока рассмотрения вторых частей заявок на участие в аукционе в электронной форме: «12» сентября 2022 Дата окончания срока рассмотрения вторых частей заявок на участие в аукционе в электронной форме: «12» сентября 2022 27. Дата подведения итогов аукциона в электронной форме Дата подведения итогов аукциона в электронной форме: «12» сентября 2022 28. Обеспечение заявок на участие в аукционе в электронной форме Не требуется 28.1. Размер обеспечения заявок на участие в аукционе в электронной форме Не требуется 28.2. Срок и порядок предоставления обеспечения заявок на участие в аукционе в электронной форме Не требуется 29. Обесп

In [59]:
#print(data_train[0][0])
doc = nlp(data_train[0][0])
print(doc.spans)



{'sc': []}


In [56]:
doc = nlp(data[0]['text'])
#print predicted spans and entity level scores
spans = doc.spans['sc']
for span, confidence in zip(spans, spans.attrs["scores"]):
    print(span.label_, confidence)

KeyError: 'scores'

In [57]:
spacy.displacy.render(doc, style="span")

Данные для обучения алгоритма spacy для задачи Span Categorization должны быть в формате JSON. Каждое предложение должно быть представлено в виде объекта, состоящего из двух полей: "text", содержащего само предложение, и "spans", содержащего список объектов, описывающих категории и местоположение соответствующих спанов в предложении. Каждый объект категории должен содержать три поля: "start", "end" и "label". Поле "start" содержит начальную позицию спана в предложении, "end" содержит конечную позицию спана в предложении, а "label" содержит название категории, к которой относится данный спан.